-sandbox
<img src="https://i1.wp.com/smittenkitchen.com/wp-content/uploads//2020/03/sk-ultimate-banana-bread.jpg?fit=1200%2C800&ssl=1" style="float:right; height:200px; padding: 10px">

# What's ~~Cooking~~ Coming in Spark 3.0 & Koalas
* Koalas
* Adaptive Query Execution
* Pandas Function APIs

In this demo, let's see which [recipes](https://www.kaggle.com/shuyangli94/food-com-recipes-and-user-interactions) people enjoy cooking!

Let's start by loading in a subset of our data with pandas.

In [3]:
import pandas as pd

pd_df = pd.read_parquet("/dbfs/summit_2020/recipes.parquet/")
pd_df.head()

-sandbox
<img src="https://www.motherjones.com/wp-content/uploads/2019/08/koala-8-20-19.jpg?w=990" style="float:right; height:200px; padding: 10px">

### Koalas

Koalas provides a rich pandas-like API on Spark, combining the best of both frameworks.

In [5]:
import databricks.koalas as ks

ks_df = ks.read_parquet("/summit_2020/recipes.parquet")
ks_df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,baked tomatoes sicilian style,59918,110,8688,2003-04-16,"['weeknight', 'time-to-make', 'course', 'main-...","[139.4, 14.0, 20.0, 2.0, 5.0, 7.0, 4.0]",11,"['preheat oven to 350', 'lightly grease a shal...",this recipe is an exercise in simplicity. don'...,"['tomatoes', 'unseasoned breadcrumbs', 'dried ...",6
1,baked tomatoes stuffed with chicken and spinach,297010,45,283251,2008-04-08,"['60-minutes-or-less', 'time-to-make', 'course...","[336.5, 20.0, 24.0, 9.0, 65.0, 32.0, 7.0]",15,"['preheat oven to 350', 'in a medium saucepan ...",from 365 ways to cook chicken.,"['onion', 'skim milk', 'flour', 'chicken bouil...",8
2,baked tomatoes stuffed with couscous and feta,51989,35,29196,2003-01-21,"['60-minutes-or-less', 'time-to-make', 'course...","[228.3, 20.0, 24.0, 9.0, 15.0, 19.0, 7.0]",10,"['preheat oven to 190 degrees c', 'bring the s...",another tasty recipe from the sunday life supp...,"['chicken stock', 'couscous', 'of fresh mint',...",9
3,baked tomatoes stuffed with orzo,253764,30,283251,2007-09-18,"['30-minutes-or-less', 'time-to-make', 'course...","[266.7, 10.0, 28.0, 42.0, 26.0, 17.0, 12.0]",13,['remove the center pulp and seeds from the to...,yum!,"['orzo pasta', 'tomatoes', 'granulated sugar',...",11
4,baked tomatoes stuffed with salmon garlic c...,132350,65,88099,2005-08-04,"['time-to-make', 'course', 'main-ingredient', ...","[280.8, 26.0, 10.0, 7.0, 49.0, 13.0, 2.0]",14,"['drain the capers if packed in vinegar', 'soa...",i was looking for different recipes to use all...,"['capers', 'salmon', 'extra virgin olive oil',...",9


### Visualization

I want to cook a recipe in under 30 minutes. How many steps do most of them involve?

In [7]:
import matplotlib.pyplot as plt

quick_recipes_df = ks_df[ks_df["minutes"] < 30]
quick_recipes_df.plot(y="n_steps", kind="hist", bins=50, title="Number of Steps")
plt.xlabel("Steps")
plt.show()

### SQL 

Let's join our dataset with the ratings table to get ratings and reviews for each recipe.

In [9]:
display(ks.sql("""
                SELECT *
                FROM ratings r
                INNER JOIN {ks_df} ON r.recipe_id = {ks_df}.id
               """))

user_id,recipe_id,date,rating,review,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
596506,38,2008-02-13,5,"Yummy, yummy, yummy! I am a big fan of fruit and yogurt together, and this was a very good mix!",low fat berry blue frozen dessert,38,1485,1533,1999-08-09,"['weeknight', 'time-to-make', 'course', 'preparation', 'occasion', 'low-protein', 'healthy', '5-ingredients-or-less', 'desserts', '1-day-or-more', 'easy', 'low-fat', 'summer', 'frozen-desserts', 'freezer', 'dietary', 'low-sodium', 'gluten-free', 'low-cholesterol', 'seasonal', 'low-saturated-fat', 'low-calorie', 'healthy-2', 'free-of-something', 'low-in-something', 'equipment']","[170.9, 3.0, 120.0, 1.0, 6.0, 6.0, 12.0]",13,"['toss 2 cups berries with sugar', 'let stand for 45 minutes , stirring occasionally', 'transfer berry-sugar mixture to food processor', 'add yogurt and process until smooth', 'strain through fine sieve', 'pour into baking pan', 'freeze uncovered until edges are solid but centre is soft', 'transfer to processor and blend until smooth again', 'return to pan and freeze until edges are solid', 'transfer to processor and blend until smooth again', 'fold in remaining 2 cups of blueberries', 'pour into plastic mold and freeze overnight', 'let soften slightly to serve']","this is yummy and low-fat, it always turns out perfect.","['blueberries', 'granulated sugar', 'vanilla yogurt', 'lemon juice']",4
596506,38,2008-02-13,5,"Yummy, yummy, yummy! I am a big fan of fruit and yogurt together, and this was a very good mix!",low fat berry blue frozen dessert,38,1485,1533,1999-08-09,"['weeknight', 'time-to-make', 'course', 'preparation', 'occasion', 'low-protein', 'healthy', '5-ingredients-or-less', 'desserts', '1-day-or-more', 'easy', 'low-fat', 'summer', 'frozen-desserts', 'freezer', 'dietary', 'low-sodium', 'gluten-free', 'low-cholesterol', 'seasonal', 'low-saturated-fat', 'low-calorie', 'healthy-2', 'free-of-something', 'low-in-something', 'equipment']","[170.9, 3.0, 120.0, 1.0, 6.0, 6.0, 12.0]",13,"['toss 2 cups berries with sugar', 'let stand for 45 minutes , stirring occasionally', 'transfer berry-sugar mixture to food processor', 'add yogurt and process until smooth', 'strain through fine sieve', 'pour into baking pan', 'freeze uncovered until edges are solid but centre is soft', 'transfer to processor and blend until smooth again', 'return to pan and freeze until edges are solid', 'transfer to processor and blend until smooth again', 'fold in remaining 2 cups of blueberries', 'pour into plastic mold and freeze overnight', 'let soften slightly to serve']","this is yummy and low-fat, it always turns out perfect.","['blueberries', 'granulated sugar', 'vanilla yogurt', 'lemon juice']",4
596506,38,2008-02-13,5,"Yummy, yummy, yummy! I am a big fan of fruit and yogurt together, and this was a very good mix!",low fat berry blue frozen dessert,38,1485,1533,1999-08-09,"['weeknight', 'time-to-make', 'course', 'preparation', 'occasion', 'low-protein', 'healthy', '5-ingredients-or-less', 'desserts', '1-day-or-more', 'easy', 'low-fat', 'summer', 'frozen-desserts', 'freezer', 'dietary', 'low-sodium', 'gluten-free', 'low-cholesterol', 'seasonal', 'low-saturated-fat', 'low-calorie', 'healthy-2', 'free-of-something', 'low-in-something', 'equipment']","[170.9, 3.0, 120.0, 1.0, 6.0, 6.0, 12.0]",13,"['toss 2 cups berries with sugar', 'let stand for 45 minutes , stirring occasionally', 'transfer berry-sugar mixture to food processor', 'add yogurt and process until smooth', 'strain through fine sieve', 'pour into baking pan', 'freeze uncovered until edges are solid but centre is soft', 'transfer to processor and blend until smooth again', 'return to pan and freeze until edges are solid', 'transfer to processor and blend until smooth again', 'fold in remaining 2 cups of blueberries', 'pour into plastic mold and freeze overnight', 'let soften slightly to serve']","this is yummy and low-fat, it always turns out perfect.

### Adaptive Query Execution (AQE)

In [11]:
spark.conf.set("spark.sql.adaptive.enabled", "true")

In [12]:
display(ks.sql("""
                SELECT *
                FROM ratings r
                INNER JOIN {ks_df} ON r.recipe_id = {ks_df}.id
               """))

user_id,recipe_id,date,rating,review,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
596506,38,2008-02-13,5,"Yummy, yummy, yummy! I am a big fan of fruit and yogurt together, and this was a very good mix!",low fat berry blue frozen dessert,38,1485,1533,1999-08-09,"['weeknight', 'time-to-make', 'course', 'preparation', 'occasion', 'low-protein', 'healthy', '5-ingredients-or-less', 'desserts', '1-day-or-more', 'easy', 'low-fat', 'summer', 'frozen-desserts', 'freezer', 'dietary', 'low-sodium', 'gluten-free', 'low-cholesterol', 'seasonal', 'low-saturated-fat', 'low-calorie', 'healthy-2', 'free-of-something', 'low-in-something', 'equipment']","[170.9, 3.0, 120.0, 1.0, 6.0, 6.0, 12.0]",13,"['toss 2 cups berries with sugar', 'let stand for 45 minutes , stirring occasionally', 'transfer berry-sugar mixture to food processor', 'add yogurt and process until smooth', 'strain through fine sieve', 'pour into baking pan', 'freeze uncovered until edges are solid but centre is soft', 'transfer to processor and blend until smooth again', 'return to pan and freeze until edges are solid', 'transfer to processor and blend until smooth again', 'fold in remaining 2 cups of blueberries', 'pour into plastic mold and freeze overnight', 'let soften slightly to serve']","this is yummy and low-fat, it always turns out perfect.","['blueberries', 'granulated sugar', 'vanilla yogurt', 'lemon juice']",4
1060485,38,2009-01-15,4,very good!! the directions were alittle weird though...,low fat berry blue frozen dessert,38,1485,1533,1999-08-09,"['weeknight', 'time-to-make', 'course', 'preparation', 'occasion', 'low-protein', 'healthy', '5-ingredients-or-less', 'desserts', '1-day-or-more', 'easy', 'low-fat', 'summer', 'frozen-desserts', 'freezer', 'dietary', 'low-sodium', 'gluten-free', 'low-cholesterol', 'seasonal', 'low-saturated-fat', 'low-calorie', 'healthy-2', 'free-of-something', 'low-in-something', 'equipment']","[170.9, 3.0, 120.0, 1.0, 6.0, 6.0, 12.0]",13,"['toss 2 cups berries with sugar', 'let stand for 45 minutes , stirring occasionally', 'transfer berry-sugar mixture to food processor', 'add yogurt and process until smooth', 'strain through fine sieve', 'pour into baking pan', 'freeze uncovered until edges are solid but centre is soft', 'transfer to processor and blend until smooth again', 'return to pan and freeze until edges are solid', 'transfer to processor and blend until smooth again', 'fold in remaining 2 cups of blueberries', 'pour into plastic mold and freeze overnight', 'let soften slightly to serve']","this is yummy and low-fat, it always turns out perfect.","['blueberries', 'granulated sugar', 'vanilla yogurt', 'lemon juice']",4
1802956262,38,2014-07-22,4,This does taste great! I think the recipe needs a bit less berries and a little bit more yogurt though. The result is still very berry!,low fat berry blue frozen dessert,38,1485,1533,1999-08-09,"['weeknight', 'time-to-make', 'course', 'preparation', 'occasion', 'low-protein', 'healthy', '5-ingredients-or-less', 'desserts', '1-day-or-more', 'easy', 'low-fat', 'summer', 'frozen-desserts', 'freezer', 'dietary', 'low-sodium', 'gluten-free', 'low-cholesterol', 'seasonal', 'low-saturated-fat', 'low-calorie', 'healthy-2', 'free-of-something', 'low-in-something', 'equipment']","[170.9, 3.0, 120.0, 1.0, 6.0, 6.0, 12.0]",13,"['toss 2 cups berries with sugar', 'let stand for 45 minutes , stirring occasionally', 'transfer berry-sugar mixture to food processor', 'add yogurt and process until smooth', 'strain through fine sieve', 'pour into baking pan', 'freeze uncovered until edges are solid but centre is soft', 'transfer to processor and blend until smooth again', 'return to pan and freeze until edges are solid', 'transfer to processor and blend until smooth again', 'fold in remaining 2 cups of blueberries', 'pour into plastic mold and freeze overnight', 'let soften slightly to serve']","this is yummy and low-fat, it always turns out perfect

### Pandas Function APIs

Let's apply a model to predict how long it will take to prepare a recipe given the number of steps and ingredients.

In [14]:
from mlflow.sklearn import load_model
from typing import Iterator
import pandas as pd

spark_df = ks_df.to_spark()

def predict(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
  model = load_model("models:/lr-model/1")
  for features in iterator:
    prediction = pd.Series(model.predict(features[["n_steps", "n_ingredients"]]), name="predicted_minutes")
    yield pd.concat([features, prediction], axis=1)

display(spark_df
        .select("name", "description", "n_steps", "n_ingredients", "minutes")
        .mapInPandas(predict, "name STRING, description STRING, n_steps INT, n_ingredients INT, minutes INT, predicted_minutes DOUBLE"))

name,description,n_steps,n_ingredients,minutes,predicted_minutes
baked tomatoes sicilian style,"this recipe is an exercise in simplicity. don't be dissuaded by the long cooking time, as these juicy italian beauties only grow more flavorful and caramelized. serve over hot cooked rice, orzo or couscous to absord all the delicious juices. from the mediterranean vegan kitchen",11,6,110,37.045648906266976
baked tomatoes stuffed with chicken and spinach,from 365 ways to cook chicken.,15,8,45,46.3153298704521
baked tomatoes stuffed with couscous and feta,another tasty recipe from the sunday life supplement of the age.,10,9,35,39.32932931103349
baked tomatoes stuffed with orzo,yum!,13,11,30,46.93889012643131
baked tomatoes stuffed with salmon garlic capers,i was looking for different recipes to use all the tomatoes this year from my garden and found this one. it's great but they are not good reheated.,14,9,65,45.96980990618273
baked tomatoes w broccoli stuffing,"this is an excellent side dish, makes a lovely presentation for a dinner party and can be fixed ahead. just be sure you don't overbake. this is from an old church cookbook.",8,9,40,36.009089013458876
baked tomatoes with a parmesan cheese crust and balsamic drizzle,there is nothing ordinary about these baked tomatoes. they are scrumptions especially at this time of the year when tomatoes are so flavorful. everything can be made up ahead of time for that last minute serving. bon appetit! update: there was some confusion whether to slice or cut the tomato in half...so i have re-worded the recipe to read a little bit better.,6,8,40,31.374248531366312
baked tomatoes with bacon and eggs,"this is a wonderful breakfast dish, and pretty enough to serve to overnight guests. serve it with toast and your favorite jam. it is really, reallly better when you have access to summer home-grown tomatoes.",15,8,40,46.3153298704521
baked tomatoes with basil orzo,nice mediterranean flavor for a light supper. could substitute yellow or green bell peppers for the tomatoes. (cut off tops and seed. omit the salting step),16,12,80,53.23385075731119
baked tomatoes with garlic tomates a la provencale,"looking for something to use up a few fresh tomatoes, and this surfaced. have yet to try it but am posting it for the zaar world tour. would be perfect with roast meat or chicken.",8,7,18,33.37988864442299


Which smoothie recipe should we recommend to Matei?

In [16]:
display(spark_df
        .filter("name like '%smoothie'")
        .orderBy(["minutes", "name"])
        .limit(1))

name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
berry berry smoothie,59998,0,68585,2003-04-17,"['15-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'for-1-or-2', 'low-protein', 'healthy', '5-ingredients-or-less', 'beverages', 'eggs-dairy', 'fruit', 'easy', 'no-cook', 'beginner-cook', 'holiday-event', 'kid-friendly', 'low-fat', 'spring', 'summer', 'winter', 'easter', 'smoothies', 'food-processor-blender', 'dietary', 'new-years', 'low-sodium', 'low-cholesterol', 'seasonal', 'low-saturated-fat', 'low-calorie', 'comfort-food', 'independence-day', 'valentines-day', 'inexpensive', 'low-in-something', 'berries', 'blueberries', 'brunch', 'superbowl', 'mardi-gras-carnival', 'taste-mood', 'sweet', 'equipment', 'small-appliance', 'number-of-servings', 'presentation', 'served-cold', 'technique']","[450.2, 4.0, 366.0, 5.0, 23.0, 7.0, 33.0]",4,"['in a blender container , combine yogurt and juice', 'blend to combine', 'add berries and process until smooth and thick', 'pour into glasses and serve immediately']","i love smoothies and make them all the time. they are so healthy, tasty and a yummy way to get that extra calcium that we all need!","['low-fat blueberry yogurt', 'grape juice', 'frozen blueberries', 'frozen blackberries']",4


The dataset used in this demo is a modified version provided by [Generating Personalized Recipes from Historical User Preferences](https://www.aclweb.org/anthology/D19-1613/), Bodhisattwa Prasad Majumder*, Shuyang Li*, Jianmo Ni, Julian McAuley. EMNLP 2019.